In [ ]:
!pip install transformers
!pip install torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Read the data set

In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/760Data/example.csv")
#df1 = pd.read_csv("example.csv")
df1['user_sentiment'].replace("Negative","0",inplace=True)
df1['user_sentiment'].replace("Positive","1",inplace=True)
df1.head()

,id,name,brand,reviews_combined,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Pink Friday: Roman Reloaded Re-Up (w/dvd),Universal Music,i love this album. it's very good. more to the...,joshua,1
1,AV14LG0R-jtxr-f38QfS,Lundberg Organic Cinnamon Toast Rice Cakes,Lundberg,Good flavor. This review was collected as part...,dorothy w,1
2,AV14LG0R-jtxr-f38QfS,Lundberg Organic Cinnamon Toast Rice Cakes,Lundberg,Good flavor. Good,dorothy w,1
3,AV16khLE-jtxr-f38VFn,K-Y Love Sensuality Pleasure Gel,K-Y,I read through the reviews on here before look...,rebecca,0
4,AV16khLE-jtxr-f38VFn,K-Y Love Sensuality Pleasure Gel,K-Y,My husband bought this gel for us. The gel cau...,walker557,0


In [ ]:
df2_train_data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/760Data/Books_train.csv")
df2_train_data['user_sentiment'].replace("Negative","0",inplace=True)
df2_train_data['user_sentiment'].replace("Positive","1",inplace=True)
df2_train_data.head()

,reviews_text,user_sentiment,id,text_clean,text_len
0,Come! Enter this Magical World! When I picked ...,1,A288WWXU3NF5QO,come enter this magical world when i picked up...,207
1,Item as described. Delivery as promised. Item ...,1,A3505BUYB2A5YR,item as described delivery as promised item as...,12
2,Four Stars My son read it and enjoyed it throu...,1,A23PTBIFGG4J7V,four stars my son read it and enjoyed it throu...,10
3,"Five Stars Great, thank you!",1,A30KZ6D730TK27,five stars great thank you,5
4,Five Stars Love the whole series of books.,1,A1J2JM2HCEOKUN,five stars love the whole series of books,8


In [ ]:
# 合并数据集
all_data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/760Data/Books_5.csv")
all_data['reviews_text'] = all_data['summary'] + ' ' + all_data['reviewText']
all_data.head()

,overall,reviewerID,asin,reviewerName,reviewText,summary,user_sentiment,reviews_text
0,5,A1REUF3A1YCPHM,0001713353,TW Ervin II,"The King, the Mice and the Cheese by Nancy Gur...",A story children will love and learn from,Positive,A story children will love and learn from The ...
1,5,A324TTUBKTN73A,0001713353,Tekla Borner,My students (3 & 4 year olds) loved this book!...,Five Stars,Positive,Five Stars My students (3 & 4 year olds) loved...
2,5,A2RE7WG349NV5D,0001713353,Deborah K Woroniecki,LOVE IT,Five Stars,Positive,Five Stars LOVE IT
3,5,A3H9YD6K9TVKDP,0001713353,James Charnock,Over and over the king has problems. Fortunat...,Not Nice Mice,Positive,Not Nice Mice Over and over the king has probl...
4,5,A3M314LI9OYME,0001713353,Jaleta Clegg,I remember reading my parents' copy of this bo...,One of my favorite kids' stories,Positive,One of my favorite kids' stories I remember re...


In [ ]:
# 使用left连接并基于两个条件将reviewText列添加到df2_train_data
df2_train_data = df2_train_data.merge(all_data[['reviews_text','asin']],
                                      left_on=['reviews_text'],
                                      right_on=['reviews_text'],
                                      how='left')



df2_train_data.head()

,reviews_text,user_sentiment,id,text_clean,text_len,asin
0,Come! Enter this Magical World! When I picked ...,1,A288WWXU3NF5QO,come enter this magical world when i picked up...,207,0006514219
1,Item as described. Delivery as promised. Item ...,1,A3505BUYB2A5YR,item as described delivery as promised item as...,12,0001048767
2,Item as described. Delivery as promised. Item ...,1,A3505BUYB2A5YR,item as described delivery as promised item as...,12,0001048767
3,Item as described. Delivery as promised. Item ...,1,A3505BUYB2A5YR,item as described delivery as promised item as...,12,0001048767
4,Item as described. Delivery as promised. Item ...,1,A3505BUYB2A5YR,item as described delivery as promised item as...,12,0001048767


In [ ]:
df2_train_data.shape

(9560, 6)

In [ ]:
df2_test_data = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/760Data/Books_test.csv")
df2_test_data['user_sentiment'].replace("Negative","0",inplace=True)
df2_test_data['user_sentiment'].replace("Positive","1",inplace=True)
df2_test_data.head()

,reviews_text,user_sentiment,id,text_clean,text_len
0,A Dance with Dragons I have read the first 4 b...,1,ATJS2S4YKKTTX,a dance with dragons i have read the first 4 b...,67
1,Good book though Had to read for school. Good...,1,A2X50137CGK1MD,good book though had to read for school good b...,11
2,Good essays Good critical essays in the back.,1,A23QB73EXAZZHW,good essays good critical essays in the back,8
3,Great book. Another hit for the Harte family s...,1,A1WT15WEGRP94L,great book another hit for the harte family se...,18
4,Five Stars awesome book..........put in bug ou...,1,A2BKG0RKN7DETW,five stars awesome bookput in bug out bag,8


In [ ]:
# 使用left连接并基于两个条件将reviewText列添加到df2_train_data
df2_test_data = df2_test_data.merge(all_data[['reviews_text','asin']],
                                      left_on=['reviews_text'],
                                      right_on=['reviews_text'],
                                      how='left')



df2_test_data.head()

,reviews_text,user_sentiment,id,text_clean,text_len,asin
0,A Dance with Dragons I have read the first 4 b...,1,ATJS2S4YKKTTX,a dance with dragons i have read the first 4 b...,67,0002247399
1,Good book though Had to read for school. Good...,1,A2X50137CGK1MD,good book though had to read for school good b...,11,0001048767
2,Good essays Good critical essays in the back.,1,A23QB73EXAZZHW,good essays good critical essays in the back,8,0001048767
3,Great book. Another hit for the Harte family s...,1,A1WT15WEGRP94L,great book another hit for the harte family se...,18,0002261367
4,Five Stars awesome book..........put in bug ou...,1,A2BKG0RKN7DETW,five stars awesome bookput in bug out bag,8,0002171856


# Using BERT's tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SentimentDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.text = dataframe[0]
        self.targets = dataframe[1].astype(int)

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())
        inputs = self.tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True,
                max_length=self.max_len,
                truncation=True,
                padding='max_length',
                return_token_type_ids=True
)
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        return {
                'ids': torch.tensor(ids, dtype=torch.long),
                'mask': torch.tensor(mask, dtype=torch.long),
                'targets': torch.tensor(self.targets[index], dtype=torch.long)
}




In [ ]:
# 划分数据集
train_df, val_df = train_test_split(df2_train_data, test_size=0.2, random_state=63)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)



In [ ]:
train_df.head()

,reviews_text,user_sentiment,id,text_clean,text_len,asin
0,"Review of The God of Small Things, by Arundhat...",1,A2KFI18C7XI2HO,review of the god of small things by arundhati...,323,0002255863
1,son-in-law hard to buy for son-in-law keeps th...,1,A1GHDMPIU3LAHN,soninlaw hard to buy for soninlaw keeps this b...,25,0002185385
2,My favorite Clancy novel. This is the story of...,1,A1VEBY2V7IHJBZ,my favorite clancy novel this is the story of ...,64,0006476414
3,Favorite book on re-read list. This is one of ...,1,A1UL4X5RJ17JJ5,favorite book on reread list this is one of my...,34,0006173624
4,Introduction This is the book everyone knows b...,1,A3OUGDEUS3EIF,introduction this is the book everyone knows b...,24,0006172768


In [ ]:
# 单独提取训练的特征和标签
train_data = train_df[["text_clean", "user_sentiment"]]
val_data = val_df[["text_clean", "user_sentiment"]]
train_data.columns = [0,1]
val_data.columns = [0,1]

In [ ]:
train_dataset = SentimentDataset(train_data, tokenizer, 128)
val_dataset = SentimentDataset(val_data, tokenizer, 128)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

# Train the model

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2).to(torch.device("cuda"))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss().to(torch.device("cuda"))


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def train_epoch(model, data_loader, loss_fn, optimizer, device):
    model = model.train()
    losses = []
    for d in data_loader:
        input_ids = d["ids"].to(device)
        attention_mask = d["mask"].to(device)
        targets = d["targets"].to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs[0], dim=1)
        loss = loss_fn(outputs[0], targets)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return sum(losses) / len(losses)

for epoch in range(4):  # 可以根据需要调整epoch数量
    train_loss = train_epoch(model, train_loader, loss_fn, optimizer, torch.device("cuda"))
    print(f"Epoch {epoch + 1}/{4} | Train loss: {train_loss}")


Epoch 1/4 | Train loss: 0.2262442125393288
Epoch 2/4 | Train loss: 0.1219598818654399
Epoch 3/4 | Train loss: 0.061914911517526594
Epoch 4/4 | Train loss: 0.031765783630647824


# Evaluation Models

In [ ]:
def eval_model(model, data_loader, device):
    model = model.eval()
    predictions = []
    true_labels = []

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["ids"].to(device)
            attention_mask = d["mask"].to(device)
            targets = d["targets"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs[0], dim=1)

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())

    return predictions, true_labels


In [ ]:
predictions, true_labels = eval_model(model, val_loader, torch.device("cuda"))

accuracy = accuracy_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")


Accuracy: 0.9309623430962343
Recall: 0.9947027663331371
Precision: 0.9321566464423607
F1 Score: 0.9624145785876993


In [ ]:
accuracy_score(true_labels, (val_df["user_sentiment"]).astype(int))

1.0

In [ ]:
print(model.config)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.34.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



# Generate a similarity matrix

In [ ]:
val_df['prediction'] = pd.Series(predictions)
val_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/760Data/predicton_MV.csv', index=False)

In [ ]:
val_df.head()

,reviews_text,user_sentiment,id,text_clean,text_len,asin,prediction
0,Excellent Thomas Nelson Publishers has done a ...,1,A3BYW6CKNH9W1M,excellent thomas nelson publishers has done a ...,52,0006064922,1
1,Another great novel by Patterson!!! There is a...,1,A3G2W8WUYKTW8N,another great novel by patterson there is alwa...,192,0006498523,1
2,Great book I gave this book a four out of five...,1,A3QV9CZN0OSV9W,great book i gave this book a four out of five...,26,0002259842,1
3,Love Perry but this attempt at action/humor fa...,0,A3ORJYOVPWJT8H,love perry but this attempt at actionhumor fal...,60,0006173586,0
4,Wonderful Non-fiction for Animal Lovers So exc...,1,A2OZMNWO2G64NK,wonderful nonfiction for animal lovers so exci...,23,0002163578,1


In [ ]:
val_df['user_sentiment'] = val_df['user_sentiment'].astype(int)
val_df['user_sentiment']

0       1
1       1
2       1
3       0
4       1
       ..
1907    1
1908    1
1909    0
1910    1
1911    1
Name: user_sentiment, Length: 1912, dtype: int64

In [ ]:
df_reco = val_df[["asin", "prediction","user_sentiment","id"]]
df_reco_clean = df_reco.dropna(subset=['user_sentiment'])
df_reco_clean['user_sentiment'] =df_reco_clean['user_sentiment'].map({0:-1,1:1})
df_reco_clean['prediction'] =df_reco_clean['prediction'].map({0:-1,1:1})
df_reco_clean['user_sentiment'].unique()
df=df_reco_clean.sample(n=500, random_state=63)

df_pivot1 = df.pivot_table(index='id', columns='asin', values='prediction', aggfunc='max')
# 使用fillna填充缺失值为0
df_pivot1 = df_pivot1.fillna(0)

In [ ]:
df

,asin,prediction,user_sentiment,id
299,0002213311,1,1,A1K4S4MWXI9E9M
423,0001844423,1,1,A3PA4NEEEZSV0L
362,0002051850,1,1,AKB010PC6G7KZ
1012,0006476155,1,1,AWPN0LDV9K9BR
1722,0001048767,1,1,A2QV1EIC3CAWRA
...,...,...,...,...
271,0002247399,1,1,A9XX3ZVJ57M0J
784,0006513255,1,1,A26YL9F3Q0SD3F
347,0001712713,1,1,A1WPAPAR89XHJJ
1107,0001844423,1,1,A3IFT6I0MVDB34


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
m1_similarity = cosine_similarity(df_pivot1)
#for elem in m1_similarity[0]:
#  print(elem)
m1_similarity.shape
sim_df = pd.DataFrame(m1_similarity)
sim_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/760Data/similarity_matrix.csv', header=True)
df_pivot1.to_csv('/content/drive/MyDrive/Colab_Notebooks/760Data/data_frame.csv', header=True)

In [ ]:
# 打印结果
df_pivot1.shape

(495, 154)

In [ ]:
df_pivot2 = df.pivot_table(index='id', columns='asin', values='user_sentiment', aggfunc='max')

# 使用fillna填充缺失值为0
df_pivot2 =df_pivot2.fillna(0)

# 打印结果
df_pivot2

asin,0001046314,0001046519,0001047868,0001048767,0001054783,0001381733,0001384198,0001712713,0001712764,0001712799,...,0006513050,0006513220,0006513255,0006513778,0006514677,0006514715,0006530699,0006531237,0006551807,0006552110
id,,,,,,,,,,,,,,,,,,,,,
A10E9PYCJPFO2S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10KYAZ7TVNP28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10S2QPKRYJT47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A115MZZUS4VVM5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A11FWYJ62IMCDX,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AYEQSGG12IFBU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AYJ2OZDJMSPEF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZEMDXXQDIQRC,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:

from sklearn.metrics.pairwise import cosine_similarity
m2_similarity = cosine_similarity(df_pivot2)
#for elem in m1_similarity[0]:
#  print(elem)
m2_similarity.shape
sim_df = pd.DataFrame(m2_similarity)
sim_df.to_csv('/content/drive/MyDrive/Colab_Notebooks/760Data/similarity_matrix2.csv', header=True)
df_pivot2.to_csv('/content/drive/MyDrive/Colab_Notebooks/760Data/data_frame2.csv', header=True)
df_pivot2.shape

(495, 154)